# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra code. 

## The Workspace directory contains the event_data files.  
## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of the appearance of the denormalized data in the file <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [5]:
# Connect to a Cassandra instance on the local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Get a session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
except Exception as e:
    print (e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print (e)


<code style="background : #ADD8E6; color : black">
    <h3> Queries are created to pose the following three questions of the data</br>
    <code style="color:red">Each Database table is modeled on a specific query that will be run.</red></h3>  

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
</code>

#### <font color=blue>Query 1 : Provide the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4</font>

In [8]:
## Create table for query 1

query1 = "CREATE TABLE IF NOT EXISTS artist_song_itemInSession "
query1 = query1 + "(session_id INT, item_in_session INT, artist TEXT, song_title TEXT, song_length FLOAT, PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query1)
except Exception as e:
    print (e)

### <font color="blue">CSV file setup and Insert Statement for Query 1</font>

In [9]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Query 1 : INSERT statement
        query1 = "INSERT INTO artist_song_itemInSession (session_id, item_in_session, artist, song_title, song_length) "
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
      
        ## column position and column name
        ## col0 = artist; col3 = itemInSession; col5=length; col8 = sessionId, col9 = song
        try:
           session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print (e)

#### <font color="blue">SELECT statement for Query 1 to verify data ingestion</font>

In [10]:
## SELECT statement verifies that the data was entered into the table
query1 = "select artist, song_title, song_length FROM artist_song_itemInSession WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print (e)
    
## use a dataframe iwth input parameters - rows (for query 1 output) and column names for the output    
df_query1 = pd.DataFrame(list(rows), columns=['Artist', 'Song Title', 'Song Length'])
df_query1.style.set_properties(**{'background-color' : '#ADD8E6', 'color' : 'black'})

,Artist,Song Title,Song Length
0,Faithless,Music Matters (Mark Knight Dub),495.307


#### <font color="blue">Query 2: Provide only the following : the name of artist, song (sorted by itemInSession), and user (first and last name) for userId=10 and sessionId = 182</font>

In [11]:
## Query 2 - CREATE TABLE

query2 = "CREATE TABLE IF NOT EXISTS artist_song_session "
query2 = query2 + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song_title TEXT, firstname_user TEXT, lastname_user TEXT, \
              PRIMARY KEY((user_id, session_id), item_in_session))"      

try:
    session.execute(query2)
except Exception as e:
    print (e)
  

#### <font color="blue">CSV file setup and Query 2 Insert Statement</font>

In [12]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
    # Query 2 : INSERT statement
        query2 = "INSERT INTO artist_song_session (user_id, session_id, item_in_session, artist, song_title, firstname_user, lastname_user) "
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"      
        
        ## column positions and column names
        ## col0 = artist; col1= firstName; col3 = itemInSession; col4 = lastName; col5=length; col8 = sessionId; col9 = song; col10 = userId
        try:
            session.execute(query2, ((int(line[10])), (int(line[8])), (int(line[3])), line[0], line[9], line[1], line[4]))     
        except Exception as e:
            print (e)

#### <font color="blue">Select Statement for Query 2 to verify data ingestion</font>

In [13]:
## Select statement verifies that the data was entered into the table
query2 = "SELECT artist, item_in_session, song_title, firstname_user, lastname_user FROM artist_song_session WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print (e)

# Use a dataframe with input parameters - rows (for query 2 output) and column names for the output 
df_query2 = pd.DataFrame(list(rows), columns=['artist', 'itemInSession', 'Song Title', 'Firstname', 'Lastname'])
df_query2.style.set_properties(**{'background-color' : '#ADD8E6', 'color' : 'black'})

,artist,itemInSession,Song Title,Firstname,Lastname
0,Down To The Bone,0,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,1,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,2,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,3,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz


### <font color="blue">Query 3: Provide every user (first and last name) in my music app history who listened to the song 'All Hands against His Own'</font>

In [14]:
# CREATE TABLE

query3 = "CREATE TABLE IF NOT EXISTS user_song_preference "
query3 = query3 + "(song_title TEXT, user_id INT, firstname_user TEXT, lastname_user TEXT, PRIMARY KEY(song_title, user_id))"

try:
    session.execute(query3)
except Exception as e:
    print (e)
                    

#### <font color="blue">CSV file setup and Query 3 Insert Statement</font>

In [15]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## INSERT statement for Query 3
        query3 = "INSERT INTO user_song_preference (song_title, user_id, firstname_user, lastname_user) "
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        
        ## col1= firstName; col4 = lastName; col9 = song; col10 = userid
      
        try:
           session.execute(query3, (line[9], int(line[10]), line[1], line[4]))           
        except Exception as e:
            print (e)

### <font color="blue">Select Statement for Query 3 to verify data ingestion</font>

In [16]:
## SELECT statement to verify the data was entered into the table
query3 = "select firstname_user, lastname_user, song_title FROM user_song_preference WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print (e)

## Use a dataframe with input parameters - the rows (for query3 output) along with column names for the output
df_query3 = pd.DataFrame(list(rows), columns=['User Firstname', 'User Lastname', 'Song Title'])
df_query3.style.set_properties(**{'background-color' : '#ADD8E6', 'color' : 'black'})

,User Firstname,User Lastname,Song Title
0,Jacqueline,Lynch,All Hands Against His Own
1,Tegan,Levine,All Hands Against His Own
2,Sara,Johnson,All Hands Against His Own


### Drop the tables before closing out the sessions

In [17]:
## Drop the table, music_app_history before closing out the sessions
query = "DROP TABLE IF EXISTS artist_song_itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print (e)
    
## Drop the table, music_app_artist_user
query2 = "DROP TABLE IF EXISTS artist_song_session"
try:
    rows = session.execute(query2)
except Exception as e:
    print (e)
    
## Drop the table, music_app_user_history
query3 = "DROP TABLE IF EXISTS user_song_preference"
try:
    rows = session.execute(query3)
except Exception as e:
    print (e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()